In [107]:
import requests
import os
import shutil

In [108]:
years = list(range(1986,2023))

In [109]:
mvp_url =  "https://www.basketball-reference.com/awards/awards_{}.html"

for year in years:
    url = mvp_url.format(year)
    
    data = requests.get(url)
    
    with open("mvp/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(data.text)

In [110]:
from bs4 import BeautifulSoup
import pandas as pd

In [112]:
dfs = []
for year in years:
    with open("mvp/{}.html".format(year), encoding="utf-8") as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="over_header").decompose()
    mvp_table = soup.find(id="mvp")
    mvp_df = pd.read_html(str(mvp_table))[0]
    mvp_df["Year"] = year
    dfs.append(mvp_df)
    
mvps = pd.concat(dfs)

mvps.tail()   

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
7,8,Stephen Curry,33,GSW,0.0,4.0,1000,0.004,64,34.5,...,5.2,6.3,1.3,0.4,0.437,0.380,0.923,8.0,0.173,2022
8,9,Chris Paul,36,PHO,0.0,2.0,1000,0.002,65,32.9,...,4.4,10.8,1.9,0.3,0.493,0.317,0.837,9.4,0.210,2022
9,10T,DeMar DeRozan,32,CHI,0.0,1.0,1000,0.001,76,36.1,...,5.2,4.9,0.9,0.3,0.504,0.352,0.877,8.8,0.154,2022
10,10T,Kevin Durant,33,BRK,0.0,1.0,1000,0.001,55,37.2,...,7.4,6.4,0.9,0.9,0.518,0.383,0.910,8.4,0.198,2022
11,10T,LeBron James,37,LAL,0.0,1.0,1000,0.001,56,37.2,...,8.2,6.2,1.3,1.1,0.524,0.359,0.756,7.5,0.172,2022


In [113]:
mvps.to_csv("mvps.csv")

In [114]:
player_pergame_stats = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

for year in years:
    url = player_pergame_stats.format(year)
    
    data = requests.get(url)
    
    with open("player_stats/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(data.text)

In [115]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [116]:
driver = webdriver.Chrome(executable_path=r"C:\Users\stnya\Downloads\chromedriver_win32\chromedriver.exe")

<ipython-input-116-3f03f1cca108>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\stnya\Downloads\chromedriver_win32\chromedriver.exe")


In [117]:
for year in years:
    url = player_pergame_stats.format(year)
    
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(2)
    
    with open("player_stats/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(driver.page_source)

In [128]:
dfs = []
for year in years:
    with open("player_stats/{}.html".format(year), encoding="utf-8") as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    tHeads = soup.findAll('tr', class_="thead")
    for tHead in tHeads:
        tHead.decompose()
    player_table = soup.find_all(id="per_game_stats")
    player_stats = pd.read_html(str(player_table))[0]
    player_stats["Year"] = year
    dfs.append(player_stats)
    
player_stats = pd.concat(dfs)

In [131]:
player_stats.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Kareem Abdul-Jabbar*,C,38,LAL,79,79,33.3,9.6,16.9,...,1.7,4.4,6.1,3.5,0.8,1.6,2.6,3.1,23.4,1986
1,2,Alvan Adams,C,31,PHO,78,45,25.7,4.4,8.7,...,1.9,4.2,6.1,4.2,1.3,0.6,2.6,3.5,10.8,1986
2,3,Michael Adams,PG,23,SAC,18,0,7.7,0.9,2.4,...,0.1,0.2,0.3,1.2,0.5,0.1,0.6,0.5,2.2,1986
3,4,Mark Aguirre,SF,26,DAL,74,73,33.8,9.0,17.9,...,2.4,3.6,6.0,4.6,0.8,0.2,3.4,3.1,22.6,1986
4,5,Danny Ainge,SG,26,BOS,80,78,30.1,4.4,8.8,...,0.6,2.4,2.9,5.1,1.2,0.1,1.6,2.6,10.7,1986


In [133]:
player_stats.tail()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
807,601,Thaddeus Young,PF,33,TOR,26,0,18.3,2.6,5.5,...,1.5,2.9,4.4,1.7,1.2,0.4,0.8,1.7,6.3,2022
808,602,Trae Young,PG,23,ATL,76,76,34.9,9.4,20.3,...,0.7,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4,2022
809,603,Omer Yurtseven,C,23,MIA,56,12,12.6,2.3,4.4,...,1.5,3.7,5.3,0.9,0.3,0.4,0.7,1.5,5.3,2022
810,604,Cody Zeller,C,29,POR,27,0,13.1,1.9,3.3,...,1.9,2.8,4.6,0.8,0.3,0.2,0.7,2.1,5.2,2022
811,605,Ivica Zubac,C,24,LAC,76,76,24.4,4.1,6.5,...,2.9,5.6,8.5,1.6,0.5,1.0,1.5,2.7,10.3,2022


In [130]:
player_stats.to_csv("player_stats.csv")

In [134]:
team_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [135]:
for year in years:
    url = team_url.format(year)
    
    data = requests.get(url)
    
    with open("team_stats/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(data.text)

In [140]:
dfs = []
for year in years:
    with open("team_stats/{}.html".format(year), encoding="utf-8") as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    east_table = soup.find_all(id="divs_standings_E")[0]
    east_df = pd.read_html(str(east_table))[0]
    east_df["Year"] = year
    east_df["Team"] = east_df["Eastern Conference"]
    del east_df["Eastern Conference"]
    dfs.append(east_df)
    
    west_table = soup.find_all(id="divs_standings_W")[0]
    west_df = pd.read_html(str(west_table))[0]
    west_df["Year"] = year
    west_df["Team"] = west_df["Western Conference"]
    del west_df["Western Conference"]
    dfs.append(west_df)

team_stats = pd.concat(dfs)    

In [141]:
team_stats.tail()

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
13,56,26,.683,—,115.6,109.9,5.37,2022,Memphis Grizzlies*
14,52,30,.634,4.0,108.0,104.7,3.12,2022,Dallas Mavericks*
15,36,46,.439,20.0,109.3,110.3,-0.84,2022,New Orleans Pelicans*
16,34,48,.415,22.0,113.2,113.0,0.02,2022,San Antonio Spurs
17,20,62,.244,36.0,109.7,118.2,-8.26,2022,Houston Rockets


In [143]:
team_stats.to_csv("team_stats.csv")